In [1]:
import pandas as pd
from openai import OpenAI
import gradio as gr

from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document
import os
import numpy as np
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_ollama import ChatOllama
from langchain.prompts import PromptTemplate
import warnings
from langchain_core._api import LangChainDeprecationWarning
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import LLMChain

In [2]:
df = pd.read_csv("data\\large_skills_dataset.csv")
df.head()

,Specialization,Skills Required
0,SEO Consultant,"Hadoop, Agile, GCP, Spring Boot, Scrum, Redis,..."
1,SEO Architect,"PostgreSQL, Vue.js, Penetration Testing, Tenso..."
2,E-commerce Consultant,"MySQL, Git, Node.js, Big Data, Smart Contracts..."
3,Bioinformatics Specialist,"Linux, TensorFlow, Scikit-learn, MySQL, Data V..."
4,Cloud Specialist,"Firebase, Data Visualization, Deep Learning, A..."


In [3]:
model = "llama3.2"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [4]:
# def get_system_prompt(skills_required):
#     system_prompt = "You are a career guidance assistant.\n"
#     system_prompt += "When the user asks about a career path, first ask them politely about their current skills.\n"
#     system_prompt += "After they provide their skills, recommend the missing skills based on their desired career.\n"
#     system_prompt += "Explain how they can learn these missing skills in a simple and friendly way.\n"
#     if skills_required:
#         system_prompt += f"The important skills for this career are: {skills_required}.\n"
#         system_prompt += "But do not assume they have any skills yet. Ask them first!\n"
#         system_prompt += "Only after they tell their skills, subtract their known skills and suggest only the missing ones.\n"
#         system_prompt += "If they say they have no skills, suggest all skills.\n"
#     return system_prompt

In [5]:
# def chat(msg, history):
#     msgs = [{"role": "system", "content": get_system_prompt("")}] + history + [{"role": "user", "content": msg}]
#     stream = openai.chat.completions.create(
#         model=model,
#         messages=msgs,
#         stream=True
#     )

#     res=""
#     for chunk in stream:
#         res += chunk.choices[0].delta.content or ""
#         yield res

In [6]:
# gr.ChatInterface(fn=chat, type="messages").launch(inbrowser=True)

In [7]:
# len(df)

In [8]:
# df_temp = df[:10]
# df_temp

In [9]:
# def get_skills(str):
#     for i in range(len(df_temp)):
#         specs = df_temp.loc[i, "Specialization"]
#         if(specs in str):
#             return df_temp.loc[df_temp['Specialization'] == specs, 'Skills Required'].values[0]

In [10]:
# def chat1(msg, history):
#     msgs = [{"role": "system", "content": get_system_prompt(get_skills(msg))}] + history + [{"role": "user", "content": msg}]
#     stream = openai.chat.completions.create(
#         model=model,
#         messages=msgs,
#         stream=True
#     )

#     res=""
#     for chunk in stream:
#         res += chunk.choices[0].delta.content or ""
#         yield res

In [11]:
# gr.ChatInterface(fn=chat1, type="messages").launch(inbrowser=True)

In [12]:
specs = [f"Specialization: {row['Specialization']}\nSkills Required: {row['Skills Required']}" for _, row in df.iterrows()]

In [13]:
# print(specs[5])

In [14]:
docs = []
for spec in specs:
    parts = spec.split("\n")
    specialization = parts[0].replace("Specialization: ", "").strip()
    skills = parts[1].replace("Skills Required: ", "").strip()

    doc = Document(
        page_content=spec,  # Full text
        metadata={
            "specialization": specialization,
            "skills": skills
        }
    )
    docs.append(doc)

texts = [doc.page_content for doc in docs]

In [18]:
# docs[0], texts[0], specs[0]

In [16]:
# db_name="vector_db"

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# vectors = embeddings.embed_documents(texts)

# if os.path.exists(db_name):
#     Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# vectorstore = Chroma.from_documents(
#     documents=docs,
#     embedding=embeddings,
#     persist_directory=db_name,
# )

# print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [17]:
# print(f"Vector for {vectorstore._collection.get(limit=1)["documents"]} is\n\n{vectorstore._collection.get(limit=1)["embeddings"]}")

In [18]:

# collection = vectorstore._collection
# result = collection.get(include=['embeddings', 'documents', 'metadatas'])
# vectors = np.array(result['embeddings'])
# print(vectors[:5])

In [19]:
model

'llama3.2'

In [20]:
# warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)
# llm = ChatOllama(model=model, temperature=0.7)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# qa_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# You are an assistant.

# Use ONLY the CONTEXT below to answer the QUESTION.
# - Do use any outside knowledge if there is no CONTEXT.

# CONTEXT:
# {context}

# QUESTION:
# {question}

# STRICT ANSWER:
# """
# )

# conversation_chain = ConversationalRetrievalChain.from_llm(
#     llm=llm,
#     retriever=retriever,
#     memory=memory,
#     combine_docs_chain_kwargs={"prompt": qa_prompt},
# )

In [21]:
# def chat2(message, history):
#     docs = retriever.get_relevant_documents(message)
#     context = "\n\n".join(doc.page_content for doc in docs)

#     prompt = f"""
# Use ONLY the following CONTEXT to answer the QUESTION.

# CONTEXT:
# {context}

# QUESTION:
# {message}

# If you don't find the CONTEXT, then you can respond with your knowledge

# STRICT ANSWER:
# """
#     result = llm.invoke(prompt)
#     return result.content

In [22]:
# if memory.chat_memory.messages:
#     memory.chat_memory.clear()
    
# view = gr.ChatInterface(chat2, type="messages").launch(inbrowser=True)

In [20]:
it = pd.read_csv("data\\encoded-IT_Job_Roles_Skills.csv")
it.head()

,Job Title,Job Description,Skills,Certifications
0,Admin Big Data,Responsible for managing and overseeing big da...,"Hadoop, Spark, MapReduce, Data Lakes, Data War...","Cloudera Certified Professional (CCP), Hortonw..."
1,Ansible Operations Engineer,Focuses on automating IT processes using Ansib...,"Ansible, Linux, Automation, Cloud Platforms, C...",Red Hat Certified Specialist in Ansible Automa...
2,Artifactory Administrator,Manages the Artifactory repository for build a...,"Artifactory, CI/CD, Jenkins, Docker, Maven, Gr...","JFrog Artifactory Certification, DevOps Instit..."
3,Artificial Intelligence / Machine Learning Leader,"Leads AI/ML projects and teams, defining strat...","AI Strategy, Machine Learning, Team Management...","AI-900: Microsoft Azure AI Fundamentals, Certi..."
4,Artificial Intelligence / Machine Learning Sr....,Senior role overseeing multiple AI/ML initiati...,"AI Strategy, Machine Learning, Team Management...",Certified Artificial Intelligence Practitioner...


In [21]:
it = it.drop(['Certifications'], axis=1)
it.head()

,Job Title,Job Description,Skills
0,Admin Big Data,Responsible for managing and overseeing big da...,"Hadoop, Spark, MapReduce, Data Lakes, Data War..."
1,Ansible Operations Engineer,Focuses on automating IT processes using Ansib...,"Ansible, Linux, Automation, Cloud Platforms, C..."
2,Artifactory Administrator,Manages the Artifactory repository for build a...,"Artifactory, CI/CD, Jenkins, Docker, Maven, Gr..."
3,Artificial Intelligence / Machine Learning Leader,"Leads AI/ML projects and teams, defining strat...","AI Strategy, Machine Learning, Team Management..."
4,Artificial Intelligence / Machine Learning Sr....,Senior role overseeing multiple AI/ML initiati...,"AI Strategy, Machine Learning, Team Management..."


In [22]:
jobdetails_doc=[]

for _, row in it.iterrows():
    doc=Document(
        page_content=f"Job Title: {row['Job Title']}\nJob Description: {row['Job Description']}\nSkills: {row['Skills']}",
        metadata={
            "Job Title": row['Job Title'],
            "Job Description": row['Job Description'],
            "Skills": row['Skills']
        }
    )

    jobdetails_doc.append(doc)

In [23]:
print(jobdetails_doc[22])

page_content='Job Title: Machine learning Architect
Job Description: Designs machine learning systems architecture. Responsible for selecting appropriate frameworks, designing workflows, and ensuring scalability of ML models.
Skills: Machine Learning, Deep Learning, Cloud Computing, Data Science, System Design, AWS, Azure, Scalability, Security, Model Deployment' metadata={'Job Title': 'Machine learning Architect', 'Job Description': 'Designs machine learning systems architecture. Responsible for selecting appropriate frameworks, designing workflows, and ensuring scalability of ML models.', 'Skills': 'Machine Learning, Deep Learning, Cloud Computing, Data Science, System Design, AWS, Azure, Scalability, Security, Model Deployment'}


In [24]:
len(jobdetails_doc)

493

In [28]:
# job_vector_directory = "jobs_vector_db"

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# if os.path.exists(job_vector_directory):
#     Chroma(persist_directory=job_vector_directory, embedding_function=embeddings).delete_collection()

# job_vectorstore = Chroma.from_documents(documents=jobdetails_doc, embedding=embeddings, persist_directory=job_vector_directory)

In [29]:
# job_vectorstore = Chroma(persist_directory=job_vector_directory, embedding_function=embeddings)
# print(job_vectorstore._collection.count())

In [30]:
# from langchain.chains import ConversationalRetrievalChain, LLMChain
# from langchain.chains.question_answering import load_qa_chain

# warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)

# llm = ChatOllama(model=model, temperature=0.7)

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# retriever = job_vectorstore.as_retriever(search_kwargs={"k": 5})

# chat_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# You are a friendly and knowledgeable career mentor. Always respond supportively and guide the user toward suitable jobs based on their skills.

# Instructions:
# 1. If the question is a greeting or casual phrase (e.g., "Hi", "Hello", "Thanks", "Thank you", "How are you?"):
#    - Respond warmly as a friend would (e.g., "Hey there! 😊 How can I help you with your career goals today?")
# 2. Otherwise:
#    - Use the job context to find matching roles.
#    - Include all jobs from vectorstore even if **only one skill** matches between the user's skills and the job's required skills, **include that job** in the response.
#    - For each matching job, provide:
#      - **Job Title**
#      - **Job Description**
#      - **Additional Required Skills**: List *all* the skills from vectorestore the user does *not* have.
#    - Encourage the user to build missing skills.
#    *DON'T REPEAT THE JOBS*
#    *Give Job Title once, Job Description once and Addition skills for that job once*
# 3. If the mentioned skill is not matching with any Skills from vectorstore then answer with your own knowledge.
# 4. Other than these two answer like: "I can't provide answer for that!"
# Context:
# {context}

# Question:
# {question}
# """
# )

# conversation_chain = ConversationalRetrievalChain.from_llm(
#     llm=llm,
#     retriever=retriever,
#     memory=memory,
#     combine_docs_chain_kwargs={"prompt": chat_prompt}
# )

In [31]:
# print(conversation_chain.invoke({"question": "I've Machine Learning as my skill, which jobs can I get from it"})["answer"])

In [32]:
# def chat3(message, history):
#     result = conversation_chain.invoke({"question": message})
#     return result["answer"]

In [33]:
# if memory.chat_memory.messages:
#     memory.chat_memory.clear()

# view = gr.ChatInterface(chat3, type="messages").launch(inbrowser=True)

In [34]:
# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# skill_vectorstore = Chroma(persist_directory="vector_db", embedding_function=embeddings)

# job_vectorstore = Chroma(persist_directory="jobs_vector_db", embedding_function=embeddings)

In [35]:
# skill_retriever = skill_vectorstore.as_retriever(search_kwargs={"k": 5})
# job_retriever = job_vectorstore.as_retriever(search_kwargs={"k": 5})

In [36]:
# skills_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# You are a helpful assistant.

# Use ONLY the CONTEXT provided below to answer the QUESTION.  
# - If there is NO CONTEXT available, you MAY use your own knowledge to respond.

# CONTEXT:
# {context}

# QUESTION:
# {question}

# STRICT ANSWER:
# """
# )

In [37]:
# jobs_prompt = PromptTemplate(
#     input_variables=["context", "question"],
#     template="""
# You are a friendly and knowledgeable career mentor. Always respond supportively and guide the user toward suitable jobs based on their skills.

# Instructions:
# 1. If the question is a greeting or casual phrase (e.g., "Hi", "Hello", "Thanks", "Thank you", "How are you?"):
#    - Respond warmly like a friend (e.g., "Hey there! 😊 How can I help you with your career goals today?")

# 2. Otherwise:
#    - Use the job context to suggest relevant job roles.
#    - Include all jobs from the vectorstore where at least **one skill** matches the user's skills.
#    - For each matching job, provide:
#      - **Job Title**
#      - **Job Description**
#      - **Additional Required Skills**: List *only* the skills required for the job that the user does *not* already have.
#    - Encourage the user to build the missing skills.

# Important:
# - **Do not repeat jobs**.
# - Provide **Job Title**, **Job Description**, and **Additional Required Skills** only once per job.

# 3. If the user's mentioned skill(s) do not match any skills from the vectorstore, respond using your own knowledge.

# 4. For all other types of questions (not related to skills or job recommendations), respond with:
#    **"I can't provide an answer for that!"**

# CONTEXT:
# {context}

# QUESTION:
# {question}
# """
# )

In [38]:
# def unified_chat(message, history):
#     category = classify_question(message)
    
#     if category == "skills":
#         docs = skill_retriever.get_relevant_documents(message)
#         context = "\n\n".join(doc.page_content for doc in docs)
#         prompt = skills_prompt.format(context=context, question=message)
#     elif category == "jobs":
#         docs = job_retriever.get_relevant_documents(message)
#         context = "\n\n".join(doc.page_content for doc in docs)
#         prompt = jobs_prompt.format(context=context, question=message)

#     result = llm.invoke(prompt)
#     return result.content

In [39]:
# if memory.chat_memory.messages:
#     memory.chat_memory.clear()

# import gradio as gr
# view = gr.ChatInterface(unified_chat, type="messages").launch(inbrowser=True)

In [26]:
# db_name="merged_db"

# embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# vectors = embeddings.embed_documents(texts)

# if os.path.exists(db_name):
#    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

# merged_vectorstore = Chroma.from_documents(
#    documents=docs,
#    embedding=embeddings,
#    persist_directory=db_name,
# )

# merged_vectorstore = Chroma.from_documents(documents=jobdetails_doc, embedding=embeddings, persist_directory=db_name)
# print(f"Vectorstore created with {merged_vectorstore._collection.count()} documents")

In [32]:
db_name="merged_db"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

merged_vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
print(f"Vectorstore created with {merged_vectorstore._collection.count()} documents")

Vectorstore created with 713 documents


In [28]:
warnings.filterwarnings("ignore", category=LangChainDeprecationWarning)

llm = ChatOllama(model=model, temperature=0.7)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retriever = merged_vectorstore.as_retriever(search_kwargs={"k": 5})

conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    # combine_docs_chain_kwargs={"prompt": merged_prompt}
)

In [29]:
message = "Jobs I can get with Google Cloud as my skill"

docs = retriever.get_relevant_documents(message)
context = "\n\n".join(doc.page_content for doc in docs)
print(context)

Specialization: Cloud Computing
Skills Required: Statistics, Git, Python, Hadoop, Adobe XD, SEO, PostgreSQL, Node.js

Specialization: Cloud Technician
Skills Required: Deep Learning, HTML, Redis, Content Marketing, Cryptography, MATLAB, Flask, Kubernetes, Linux, ElasticSearch

Specialization: Cloud Manager
Skills Required: Blockchain, SEO, NLP, Redis, PyTorch, Figma, Content Marketing, Machine Learning

Job Title: GCP DevOps Engineer
Job Description: Implements DevOps practices on Google Cloud Platform. Responsible for automating deployments, managing CI/CD pipelines, and ensuring system reliability.
Skills: Google Cloud, DevOps, CI/CD, Terraform, Ansible, Docker, Kubernetes, Linux, Automation, Scripting

Specialization: Cloud Developer
Skills Required: Kubernetes, Smart Contracts, Swift, CSS, PostgreSQL, Statistics, Angular, Computer Vision, Scrum


In [33]:
def chat4(message, history):
    docs = retriever.get_relevant_documents(message)
    context = "\n\n".join(doc.page_content for doc in docs)

    history.append({"role": "user", "content": message})
    
    conversation_history = "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])
    
    prompt = f"""
You are a friendly chatbot, guiding the user about their career.
Use ONLY the following CONTEXT to answer the QUESTION.

Provide a friendly, motivating message about the user's career goal, along with the skills or job information they are asking for.

For skills:
- If the user asks about skills, provide all skills required for the specialization that matches the query. If no specialization is found, fall back to the job title’s context and provide those skills.
- Give a one-line idea about each skill mentioned, making them excited about acquiring the skills.
Example:
A Product Engineer bridges the gap between technical development and product design, using engineering skills to create scalable, user-focused products that solve real-world problems. This role blends coding, system design, data handling, and user experience to deliver top-tier tech products from concept to deployment.

Here’s a one-line exciting pitch for each required skill:

MySQL: Learn to speak the language of databases and unlock insights that drive impactful product decisions.

TypeScript: Build robust, maintainable apps with confidence using this powerful upgrade to JavaScript.

React: Craft sleek, responsive interfaces and bring products to life with one of the most in-demand front-end frameworks.

Java: Power backend systems with a battle-tested language known for performance, scalability, and reliability.

Hadoop: Dive into the world of big data and process massive information streams like a data wizard.

Computer Vision: Give your apps the ability to "see" and revolutionize user experience through intelligent image understanding.

For jobs:
- If the user asks about jobs, provide a list of all jobs and specializations that require the specified skills.
- Also give them job description and also *skills* for that job from job title context**(mention skills right below the specified job)**
Example:
1. **Machine Learning Engineer**: Build and deploy intelligent models that will revolutionize industries. Your dream job awaits!
Skills required: Python, Machine Learning, TensorFlow, Keras, PyTorch, Data Analysis, Deep Learning, Natural Language Processing, Computer Vision, Data Science, Algorithms

Do not repeat context as it is. Instead, motivate and inspire the user with the information you provide, and encourage them to keep learning.
Don't respond like: 'from your context', 'from given context', etc

Finally motivate them in there journey

CONTEXT:
{context}

HISTORY:
{conversation_history}

QUESTION:
{message}

STRICT ANSWER:
"""

    result = llm.invoke(prompt)
    
    history.append({"role": "bot", "content": result.content})
    
    return [{"role": "bot", "content": result.content}]

if memory.chat_memory.messages:
    memory.chat_memory.clear()

view = gr.ChatInterface(chat4, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
